The objective of the following is to obtain the blog raw text & tags associated with the blogs

In [ ]:
import unicodedata
import sys


# Constants
RAW_SQL_DATA = "RAW SQL data.txt"
BLOG_TAG_FILENAME = "blog and tags.txt"
NON_ENGLISH_BLOG_TAG_FILENAME = "non-English blog and tags.txt"

# https://stackoverflow.com/questions/11066400/remove-punctuation-from-unicode-formatted-strings/11066687#11066687
PUNCTUATION_UNICODE_TABLE = dict.fromkeys(i for i in range(sys.maxunicode) if unicodedata.category(chr(i)).startswith('P'))

from nltk.corpus import stopwords
STOPWORD_LIST = stopwords.words('english') 

Library Functions

In [ ]:
from bs4 import BeautifulSoup

def removeHtmlCss(html):
    """
    Description: This function cleans up an input string by removing the HTML tags from it.
    
    Source: https://stackoverflow.com/questions/22799990/beatifulsoup4-get-text-still-has-javascript
    https://stackoverflow.com/questions/6467043/extracting-element-and-insert-a-space
    ^ I had to figure that out because URLs would be meshed up with future lines.
    e.g. "found at http://www.tritonsvc.com.</p><p><strong><em>About"
    Would become "found at http://www.tritonsvc.com.About" without the (separator=u' ')
    """
    
    soup = BeautifulSoup(html, "html5lib") # create a new bs4 object from the html data loaded

    for script in soup(["script", "style"]): # remove all javascript and stylesheet code
        script.extract()
        # script.decompose()
        
    # Adds space after removing HTML/CSS tags
    # Todo: Figure out way to put words back together
    text = soup.get_text(separator=u'')

    return text

# To install langdetect: pip install langdetect
import langdetect
from langdetect import detect
from langdetect.lang_detect_exception import ErrorCode, LangDetectException

# This function takes in an input string, and checks if most of that string is in English or not
def detectLanguage(text):
    """
    Description: This function returns a list of possible languages that an input string can be
    """
    # Detect if string contains mostly non-Latin characters
    chinese_character_count = 0
    for ch in text:
        if ord(ch) > 0x4e00 and ord(ch) < 0x9fff:
            #print("Chinese: " + str(ch))
            chinese_character_count += 1

    chinese_percentage = chinese_character_count / (len(text.split()) + chinese_character_count)
    
    if chinese_percentage > 0:
        print("Chinese Percentage: " + str(chinese_percentage) + ", " + str(title))
    
    if chinese_percentage > 0 and chinese_percentage < 0.50:
        print("Warning - Unsure if Chinese! Assumming it is not Chinese!")
        #print("Chinese Percentage: " + str(chinese_percentage) + ", " + str(title))
        print(str(text))
        
    elif chinese_percentage > 0.50:
        print("Probably Chinese")
        #print("Chinese Percentage: " + str(chinese_percentage) + ", " + str(title))
        return ['zh:{}'.format(chinese_percentage)]
    
    try:
        # Check for other foreign languages
        language_dector_result = detect(text)

        possible_language_list = langdetect.detect_langs(text)
        if len(possible_language_list) >= 2:
            print(text)
            print(possible_language_list)

        # print(language_dector_results)
        # if language_dector_results != 'en':
        #    print(language_dector_results)

        return possible_language_list

    except LangDetectException:
        print("Empty Blog Entry?")
        #print("Title: {}\nBlog: {}\nTags: {}\n".format(title_line, blog_line[:100], tag_line))
        print("Text: {}".format(text))
        print("\n\n")
        return "UNKNOWN"

def _remove_all_attrs(soup):
    for tag in soup.find_all(True): 
        tag.attrs = {}
    return soup 
    
import nltk
import string
import re
from nltk.tokenize import TweetTokenizer
    
def cleanUpText(original_text):
    """
    Description: This function removes puncutation
        
    XXX: Add flags for 
        - removing numbers
        - certain types of punctuation
        - retaining lower/uper-case
    """
    
    modified_text = original_text

    # Convert text to all lower-case
    modified_text = modified_text.lower()
    
    # Remove URLs/File directories
    modified_text = re.sub(r'^https?:\/\/.*[\r\n]*', '', modified_text, flags=re.MULTILINE)

    # Remove punctuation
    modified_text = modified_text.translate(PUNCTUATION_UNICODE_TABLE)

    # Remove numbers
    modified_text = re.sub(r'\d+', '', str(modified_text))

    # Tokenize
    # tokens = no_numbers.split()
    
    # Remove code
    
    
    
    return modified_text
    
def printMostCommonWords(tokens):
    """
    Graphs the 50 most common tokens given
    """
    
    ##############
    # Graph Data #
    ##############
    import nltk
    import matplotlib.pyplot as plt

    # This is to make the graph bigger
    plt.figure(figsize=(15, 5))  # the size you want

    # Remove stopwords according to NLTK
    filtered_words = [w for w in tokens if not w in STOPWORD_LIST]

    fdist2 = nltk.FreqDist(filtered_words)
    fdist2.plot(50, cumulative=False)
    
    print(fdist2.most_common(50))

In [ ]:
import datetime
import mysql.connector
from mysql.connector import errorcode
import xml.etree.ElementTree as ET

cnx = mysql.connector.connect(user='root', password='liferay',
                              host='127.0.0.1',
                              database='sample_customer_data')
cnx.close()

try:
    cnx = mysql.connector.connect(user='root', password='liferay',
                              host='127.0.0.1',
                              database='sample_developer_data',
                              charset='utf8')

    cursor = cnx.cursor(buffered=True)
    cursor2 = cnx.cursor()

    query = ("SELECT blogsentry.title, blogsentry.content, assetentry.entryId FROM sample_developer_data.assetentry \
                INNER JOIN sample_developer_data.blogsentry \
                ON sample_developer_data.assetentry.classPK = sample_developer_data.blogsentry.entryId;")
    cursor.execute(query)
    
    counter = 0

    #print(cursor)
    
    counter_with_content = 0
    
    file = open(BLOG_TAG_FILENAME, "w", encoding='utf8')
    non_english_file = open(NON_ENGLISH_BLOG_TAG_FILENAME, "w", encoding='utf8')

    for (title, content, entryId) in cursor:
        
        # Cleanup Text
        clean_title = removeHtmlCss(title)
        clean_title = clean_title.replace("]]>", "")
        clean_content = removeHtmlCss(content)
        clean_content = clean_content.replace("]]>", "")
        #clean_content = content
        
        # Run the HTML CSS cleanup again, because of nested HTML/CSS/JS code
        #clean_content = removeHtmlCss(clean_content)

        #print(notags)
        
        
        # Reduce all whitespace to a single space
        clean_title = " ".join(clean_title.split()).strip()
        clean_content = " ".join(clean_content.split()).strip()
                
        query2 = ("SELECT assettag.name FROM sample_developer_data.assettag \
                      WHERE sample_developer_data.assettag.tagId \
                      IN (SELECT assetentries_assettags.tagId \
                        FROM sample_developer_data.assetentries_assettags \
                        WHERE sample_developer_data.assetentries_assettags.entryId = " + str(entryId) + ")")

        cursor2.execute(query2)
        tag_list = list()
        
        for (name) in cursor2:
            tag_list.append(name[0])
        
        # Verify that tags exist & there's content
        if tag_list and clean_content:
            
            # Verify content is in English
            content_language = detectLanguage(clean_content)
            
            print(content_language)
            
            lang = str(content_language[0]).split(':')
            
            # if ('en' in content_language) and (content_language.count(':') <= 1):
            if ((lang[0] == 'en') and (float(lang[1]) > .80)):
                counter_with_content += 1

                print("\n{}) {}\n{}\n{}\n\n".format(counter, clean_title, clean_content[:50], tag_list))

                output = "{}\n{}\n{}\n\n".format(title, clean_content, ",".join(tag_list))
                output.encode('utf8')
                # print(output)
                file.write(output)
            else:
                output = "{}\n{}\n{}\n{}\n\n".format(title, clean_content, ",".join(tag_list), content_language)
                output.encode('utf8')
                non_english_file.write(output)
                continue
                
        else:
            #print("{}) Empty String".format(counter))
            #output = "{}\n{}\n{}\n\n".format(title, clean_content, ",".join(tag_list))
            pass
            
        
        #print(content[0].strip())
        counter += 1
        
        if counter % 500 == 0:
            print("Current Count: " + str(counter))
            pass

    file.close()
    non_english_file.close()
    print("Total Entries: {}\nContent Entries: {}".format(counter, counter_with_content))

except mysql.connector.Error as err:
  if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
    print("Something is wrong with your user name or password")
  elif err.errno == errorcode.ER_BAD_DB_ERROR:
    print("Database does not exist")
  else:
    print(err)
else:
  cnx.close()



This section will open the blog & tags file, and cleanup whitespace.

In [ ]:
#from langdetect import detect_langs
import langdetect
from langdetect import detect
from langdetect.lang_detect_exception import ErrorCode, LangDetectException

title_list = list()
blog_list = list()
tag_list = list()

blog_tag_file = open(BLOG_TAG_FILENAME, 'r', encoding='utf-8')
    
while True:
    title_line = blog_tag_file.readline().strip()
    blog_line = blog_tag_file.readline().strip()
    tag_line = blog_tag_file.readline().strip()
    
    try:
        #langauge = langdetect.detect_langs(blog_line)
        language = detect(blog_line)
    except LangDetectException:
        print("Empty Blog Entry?")
        print("Title: {}\nBlog: {}\nTags: {}\n".format(title_line, blog_line[:100], tag_line))
        print("\n\n")
    
    blog_tag_file.readline() # This is just to burn the extra new line
    #print("Title: {}\nBlog: {}\nTags: {}\n".format(title_line, blog_line[:100], tag_line))
    #print("Title: {} - {} {}\n".format(title_line, blog_line[100:], langauge))
    
    # We only care about entries that have blog entries
    if tag_line:
        title_list.append(title_line)
        blog_list.append(blog_line)
        tag_list.append(tag_line)
    else:
        print("End of File")
        print("Title: {}\nBlog: {}\nTags: {}\n".format(title_line, blog_line[:100], tag_line))
        break # EOF

print(blog_list[0][:100])
print(blog_list[len(title_list) - 1][:100])
print(len(title_list) - 1)

In [ ]:
# Write back to file with:
# - HTML removed
# - Remove URLs (http, https, ftp, svn)

import re

# Note: This is a very naive implementation, there is probably a better way to do this.
# This regex is to remove any URLs
regex = re.compile(r"(http|ftp|https|svn):\/\/[\S]+", re.IGNORECASE)

file1 = open('CLEANUP 1 - Remove HTML and CSS.txt', "w", encoding='utf8')
file2 = open('CLEANUP 2 - Remove URLs.txt', "w", encoding='utf8')
file3 = open('CLEANUP 3 - Remove Puncuation.txt', "w", encoding='utf8')
file4 = open('CLEANUP 4 - Remove Numbers.txt', "w", encoding='utf8')
file5 = open('CLEANUP 5 - Convert to Lower Case.txt', "w", encoding='utf8')
file6 = open('CLEANUP 6 - Adjust Whitespace.txt', "w", encoding='utf8')

for index in range(len(title_list)):
    title = title_list[index]
    blog = blog_list[index]
    tags = tag_list[index]
    
    # Cleanup blog contents

    # Remove HTML, CSS, & JS
    clean_blog = removeHtmlCss(blog)
    output_string = "{}\n{}\n{}\n\n".format(title, clean_blog, tags)
    file1.write(output_string)
    
    # Remove URLs
    clean_blog = regex.sub("", clean_blog)
    output_string = "{}\n{}\n{}\n\n".format(title, clean_blog, tags)
    file2.write(output_string)
    
    # Remove punctuation
    clean_blog = clean_blog.translate(PUNCTUATION_UNICODE_TABLE)
    output_string = "{}\n{}\n{}\n\n".format(title, clean_blog, tags)
    file3.write(output_string)
    
    # Remove numbers
    #clean_blog = re.sub(r'\d+', '', str(clean_blog))
    #output_string = "{}\n{}\n{}\n\n".format(title, clean_blog, tags)
    #file4.write(output_string)
    
    # Convert to lowercase
    clean_blog = clean_blog.lower()
    output_string = "{}\n{}\n{}\n\n".format(title, clean_blog, tags)
    file5.write(output_string)
    
    # Adjust whitespace to only single spaces
    clean_blog = " ".join(clean_blog.split()).strip()
    output_string = "{}\n{}\n{}\n\n".format(title, clean_blog, tags)
    file6.write(output_string)
    
    # output_string = "{}\n{}\n{}\n\n".format(title, clean_blog, tags)
    
    #file.write(output_string)
    
    # Update array contents
    #title_list[index] = 
    blog_list[index] = clean_blog
    #tag_list[index]

#file.close()

print(blog_list[0][:100])
print(blog_list[len(title_list) - 1][:100])
print(len(title_list) - 1)

file1.close()
file2.close()
file3.close()
file4.close()
file5.close()
file6.close()

Print most common words in all the blogs

In [ ]:
# Read in cleaned words from last file

cleaned_blog_list = list()

file6 = open('CLEANUP 6 - Adjust Whitespace.txt', "r", encoding='utf8')
    
while True:
    title_line = file6.readline().strip()
    blog_line = file6.readline().strip()
    tag_line = file6.readline().strip()
        
    file6.readline() # This is just to burn the extra new line
    #print("Title: {}\nBlog: {}\nTags: {}\n".format(title_line, blog_line[:100], tag_line))
    
    # We only care about entries that have blog entries
    if title_line:
        #title_list.append(title_line)
        cleaned_blog_list.append(blog_line)
        #tag_list.append(tag_line)
    else:
        print("End of File")
        print("Title: {}\nBlog: {}\nTags: {}\n".format(title_line, blog_line[:100], tag_line))
        break # EOF

# Tokenize
tokens = ' '.join(cleaned_blog_list).split()

# Graph words
printMostCommonWords(tokens)


Calculating frequency of words on a per document basis. (i.e. if a document is in every document, then it has the highest frequency possible.)

In [ ]:
from collections import defaultdict

# Build vocabulary set

vocab_document_count = defaultdict(lambda: 0)

for blog_words in blog_list:
    tokens = blog_words.split()
    vocab_set = set(tokens)
    
    # print('\n' + blog_words[:50])
    
    # Iterate through set, and update the vocab_document_count
    for word in vocab_set:
        vocab_document_count[word] += int(1)
        
    # Debugging: Printing all the values in the dictionary
    #for word, count in vocab_document_count.items():
    #    print("{} : {}".format(word, count))
    
    #break

# Debugging: Printing all the values in the dictionary
#for word, count in vocab_document_count.items():
#        print("{} : {}".format(word, count))
        
vocab_list = list(vocab_document_count)
word_index = {w: idx for idx, w in enumerate(vocab_document_count)}

VOCABULARY_SIZE = len(vocab_list)
DOCUMENTS_COUNT = len(blog_list)
 
print("Vocab Size: " + str(VOCABULARY_SIZE) + " Document Count: " + str(DOCUMENTS_COUNT))      # 10788, 51581

for w in sorted(vocab_document_count, key=vocab_document_count.get, reverse=True):
    print("{} : {}".format(w, vocab_document_count[w]))

Cleanup the data by removing stopwords, or haplax legomenon

In [ ]:
# Remove Low Frequency & High Frequency Words

low_frequency_word_list = list()
high_frequency_word_list = list()

LOW_FREQUENCY_LIMIT_INCLUSIVE = 4
HIGH_FREQUENCY_LIMIT_INCLUSIVE = 500

unique_words_before = len(vocab_document_count)

# Iterate through document frequency dictionary, and remove all low frequency instances
for word in list(vocab_document_count.keys()):
    
    if vocab_document_count[word] <= LOW_FREQUENCY_LIMIT_INCLUSIVE:
        del vocab_document_count[word]
        low_frequency_word_list.append(word)
        
unique_words_after = len(vocab_document_count)

print("Length (before): {}, Length (after): {}".format(unique_words_before, unique_words_after))
    
# vocab_document_count = vocab_document_count_removed_low_limit

Obtaining Q1, Q2, Q3, Q4

In [ ]:
number_of_entries = len(vocab_document_count)
max_index = number_of_entries - 1
min_index = 0
q2_index = int(number_of_entries / 2)
q1_index = int(q2_index / 2)
q3_index = q2_index + q1_index

sorted_word_list = sorted(vocab_document_count, key=vocab_document_count.get, reverse=False)

print("Min: {} ({})".format(sorted_word_list[min_index], vocab_document_count[sorted_word_list[min_index]]))
print("Q1:  {} ({})".format(sorted_word_list[q1_index], vocab_document_count[sorted_word_list[q1_index]]))
print("Q2:  {} ({})".format(sorted_word_list[q2_index], vocab_document_count[sorted_word_list[q2_index]]))
print("Q3:  {} ({})".format(sorted_word_list[q3_index], vocab_document_count[sorted_word_list[q3_index]]))
print("Q4:  {} ({})".format(sorted_word_list[max_index], vocab_document_count[sorted_word_list[max_index]]))
      
      #Max: {} ({})".format(vocab_document_count[min_index]))


In [ ]:
# Plotting the Data

#plt.bar(range(len(vocab_document_count)), vocab_document_count.values(), align='center')
#plt.xticks(range(len(vocab_document_count)), vocab_document_count.keys())

#plt.show()

# Calculate Stats on the distro
###############################
document_frequency_list = list(vocab_document_count.values())

# Calcuate Mean, Median, Stdev
import numpy as np

mean = np.mean(document_frequency_list)
median = np.median(document_frequency_list)
std_dev = np.std(document_frequency_list)

print("Mean: {}\nMedian: {}\nStandard Deviation: {}".format(mean, median, std_dev))

# Graph data
############
import matplotlib as mpl
import matplotlib.pyplot as plt 

# Clear previous instances
plt.clf()

# Create a figure instance
fig = plt.figure(1, figsize=(9, 6))

# Create an axes instance
ax = fig.add_subplot(111)

# Create the boxplot
bp = ax.boxplot(document_frequency_list)
ax.set_yscale('log')

# Save the figure
fig.savefig('Vocab Frequency by Documents.png', bbox_inches='tight')

Calculate (Inverse) Document Frequency

In [ ]:
from collections import defaultdict
word_df = defaultdict(lambda: 0)

for blog_words in blog_list:
    tokens = blog_words.split()
    word_set = set(tokens)
    
    for word in word_set:
        word_df[word] += 1

import math
for word in vocab_list:
    if word_df[word] >= 2:
        #word_df[word] = math.log((word_df[word] + 1) / len(blog_list))
        word_df[word] = word_df[word] / len(blog_list)
    
#counter = 0
#for word in word_df:
#    print("Word: {} - {}".format(word, word_df[word]))
    
#    if counter > 10:
#        break
#    else:
#        counter += 1
        
sorted_list = sorted(word_df.items(), key=lambda x:x[1])

word_list = []
log_score_list = []

raw_count_list = []

for entry in sorted_list:
    print("{} - {}".format(entry[0], entry[1]))
    word_list.append(entry[0])
    log_score_list.append(entry[1])
    
    if entry[1] >= 2:
        raw_count_list.append(entry[1])
    
#word_list = [x[0] for x in sorted_list]
#log_score_list = [x[1] for x in sorted_list]

#print(word_list[2])
#print(word_list[3])
#print(log_score_list[2:])

Calculate key statistical information

In [ ]:
# Calcuate Mean, Median, Stdev
import numpy as np

mean = np.mean(log_score_list)
median = np.median(log_score_list)
std_dev = np.std(log_score_list)

print("Mean: {}\nMedian: {}\nStandard Deviation: {}".format(mean, median, std_dev))


Attempting to graph the data

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt 

# Create a figure instance
fig = plt.figure(1, figsize=(9, 6))

# Create an axes instance
ax = fig.add_subplot(111)

# Create the boxplot
bp = ax.boxplot(log_score_list)
#bp = ax.boxplot(raw_count_list[:500])

# Save the figure
fig.savefig('fig1.png', bbox_inches='tight')

In [ ]:
clean_content = "tu madre su se your momma is so fat"            
content_language = detectLanguage(clean_content)

type(content_language)

print(content_language[0])

In [ ]:
testing = '作者 Bruno Farache英文原文 Web Content（以前叫Journal）最近发布一个新功能。'

testing.split()

In [ ]:
cleanUpText('hello. asdfljklajksdf;asfdljjlafsjioaewjr Timmy')

In [ ]:
regex = re.compile(r"(http|ftp|https):\/\/[\S]+", re.IGNORECASE)
test_string = "http://www.edward.com"


regex.sub("", test_string)

In [ ]:
removeHtmlCss('<strong>1. Load Balance</strong>r - it can be software (i.e. - Apache), or hardware (F5), or whatever you wish, really. ')

In [ ]:
language_results = detectLanguage('hello my name is huan, es muy bieno')
print(language_results)

lang = str(language_results[0]).split(':')
print(lang)